# Abstract

Try out various perfomance checks and issues

# Environment

# Setup debugging log
import logging
logger = logging.getLogger('jwst.associations')
handler = logging.StreamHandler()
logger.addHandler(handler)
handler.setLevel(logging.DEBUG)
logger.setLevel(logging.DEBUG)

In [1]:
import cProfile
import pdb

In [2]:
from jwst.associations.tests.helpers import (
    combine_pools,
    registry_level3_only
)

In [3]:
from jwst.associations import generate

# Library

# Main

# The most basic

In [4]:
pool = combine_pools('data/pool_012_image_miri_single_exp.csv')

/Users/eisenham/anaconda3/envs/jwstdevpy3/lib/python3.5/site-packages/astropy/table/table.py:1301: StringTruncateWarning: truncated right side string(s) longer than 14 character(s) during assignment
  col[item] = val


In [5]:
rules = registry_level3_only()

In [6]:
asns, orphaned = generate(pool, rules)

Calling constraints_to_text


In [7]:
cProfile.run('asns, orphaned = generate(pool, rules)')

Calling constraints_to_text
         475440 function calls (462436 primitive calls) in 0.374 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.373    0.373 <string>:1(<module>)
       24    0.000    0.000    0.000    0.000 <string>:12(__new__)
      140    0.000    0.000    0.000    0.000 __init__.py:1258(debug)
      140    0.000    0.000    0.000    0.000 __init__.py:1502(getEffectiveLevel)
      140    0.000    0.000    0.000    0.000 __init__.py:1516(isEnabledFor)
        1    0.000    0.000    0.000    0.000 __init__.py:182(dumps)
        1    0.000    0.000    0.000    0.000 __init__.py:243(load)
        1    0.000    0.000    0.000    0.000 __init__.py:271(loads)
        1    0.000    0.000    0.000    0.000 __init__.py:984(__getitem__)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:23(getpreferredencoding)
       34    0.000    0.000    0.000    0.000 _collections_abc.py:

In [ ]:
asns

In [ ]:
pdb.run("asns, orphaned = generate(pool, rules)")